In [ ]:
%run initialize/Week_1_Initialize.ipynb

# Characterization of Coastal Systems

Welcome to the first notebook exercise of Coastal Systems (TU Delft, MSc Coastal Engineering)! This is the first year that we will experiment with notebooks in this course. With these notebooks we hope to provide you with interactive course material that helps you better understand the processes and concepts that we teach in this course. Please let us know how you find the notebooks - we appreciate your feedback!

Chapter 2 of [Coastal Dynamics Open Textbook](https://textbooks.open.tudelft.nl/textbooks/catalog/view/37/92/383-1) describes the large geographical variation of coasts across the world. It explains how the coasts that we have today are shaped by both present-day processes and processes millions years ago. It distinguishes between three different order of features, which are associated to different orders of of time. In this notebook we will look at coastal systems at these different orders of scale. 

## Import libraries that we use for our analysis

In the two cells below we import the libraries that we need for the analysis. We also set some path settings to load the data and source code. For example, in the cell below we add the `src` directory to the system path, which allows us to import generic functions from `../../src/coastpy`. 

In [1]:
import os
import pathlib

import colorcet as cc
import dask.dataframe as dd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import hvplot.xarray  # noqa: API import
import ipyleaflet
import numpy as np
import pandas as pd
import panel as pn
from geoviews import tile_sources as gvts
from ipyleaflet import Map, Marker, ScaleControl, basemaps
from ipywidgets import HTML


from coastal_dynamics.geometries import geo_bbox

cwd = pathlib.Path().resolve()
proj_dir = cwd.parent  # this is the root of the CoastalCodeBook

DATA_DIR = proj_dir / "data"
coastal_systems_fp = DATA_DIR / "01_coastal_systems.gpkg"

pn.extension()

print(cwd)

C:\Users\khdeb\Projects\CoastalCodebook\notebooks


## Part 1: Tectonic classification

In this notebook we will start with the broadest (or first order) features of the coast that cover large geographical distances (thousands of kilometres) and are linked to the long-term geological process of plate tectonics. We will do so by using earthquake data from the [USGS](https://earthquake.usgs.gov/earthquakes/search/). The dataset we load contains a sample (10\%) of observed eartquakes between Jan 2000 and Dec 2018. Why earthquake data? Earthquake data reveals geologists the mysteries of the deep, but also for coastal researchers the data is insightful. Let's first load the data by running the next cells. 

### Load the earthquake data

We load the data (tabular data including geometries) and index the columns to only keep the data in memory that we actually need. In total the dataset contains 2.2 million earthquakes, but here we use a sample (10\%), so the data contains approx. 220k eartquake entries. If you find that the interactive panel responds slow to how you adjust the widgets, please consider to take another sample. You can do so by uncommenting the sample line in the next cell. So if you set frac=0.1 you have a dataframe with approx. 22k unique earthquakes over the world. 
 

In [ ]:
WEB_MERCATOR_LIMITS = (
    -20037508.342789244,
    20037508.342789244,
)  # max polar latitudes that can be handled in World Mercator

df = (
    dd.read_parquet(DATA_DIR / "01_earthquakes_sample.parquet")
    .sample(
        frac=0.1
    )  # uncomment this line if loading the data takes too long on your computer
    .set_index("time")
    .compute()
    .tz_localize(None)
    .sort_index()
)


# To save memory we drop most of the columns. Also we drop the polar latitudes that cannot be displayed in the web mercator projection.
df = df[["mag", "depth", "latitude", "longitude", "place", "type"]][
    df["northing"] < WEB_MERCATOR_LIMITS[1]
]
# df.head()

### Visualization of the earthquake data

To explore the data we use visualization tools from the [Holoviz project](https://holoviz.org/) that makes high-level tools to simplify visualization in Python. In the next cell we enable the interactive mode on the data dataframe, create widgets to explore the data and filter the dataframe accordingly. To explore the eartquake data we create an overlay of the eartquakes on a tileset of ESRI Imagery. Please note that the code in the next cell will only do the computations and store the result in an object called `panel`. To actually see the results you have to run one more cell; the one that calls this object panel. 



In [ ]:
from bokeh.models import PanTool, WheelZoomTool

In [ ]:
pn.extension()

# title_bar =  pn.pane.Markdown(
#         "##Exercise 1: Plate tectonics & first-order coastal features",
#         styles={"color": "black"},
#         width=800,
#         align='center'
#         # margin=(10, 5, 10, 15),
#     )

# define widgets that can be used to index the data
magnitude_slider = pn.widgets.RangeSlider(
    name="Earthquake magnitude [Richter]", start=0.1, end=10
)
depth_slider = pn.widgets.RangeSlider(name="Earthquake depth [km]", start=0.1, end=650)
date_slider = pn.widgets.DateRangeSlider(
    name="Date", start=df.index[0], end=df.index[-1]
)
column_types = pn.widgets.Select(options=["mag", "depth"])


@pn.depends(
    magnitude_slider.param.value_start,
    magnitude_slider.param.value_end,
    depth_slider.param.value_start,
    depth_slider.param.value_end,
    date_slider.param.value_start,
    date_slider.param.value_end,
    column_types.param.value,
)
def plot_earthquake_panel(
    magnitude_start,
    magnitude_end,
    depth_start,
    depth_end,
    date_start,
    date_end,
    column_type,
):

    panel = df[
        (df.mag > magnitude_start)
        & (df.mag < magnitude_end)
        & (df.depth > depth_start)
        & (df.depth < depth_end)
        & (df.index >= date_start)
        & (df.index <= date_end)
    ]
    # inverted fire colormap from colorcet
    cmap = cc.CET_L4[::-1]
    colorbar_labels = {"mag": "Magnitude [Richter]", "depth": "Earthquake depth [km]"}

    p = panel.hvplot.points(
        x="longitude",
        y="latitude",
        geo=True,
        color=column_type,
        global_extent=True,
        tiles="ESRI",
        # frame_width=900,
        ylabel="Latitude [deg]",
        xlabel="Longitude [deg]",
        cmap=cmap,
        tools=["tap"],
        hover_cols=["place", "time"],
        logz=True,
        clim=(1, None),
        clabel=colorbar_labels[column_type],
    )

    p.opts(width=1000, height=500, tools=['wheel_zoom'])
    
    return p

earthquake_panel = pn.Column(
    # pn.Row(title_bar, align='center'),
    pn.Row(column_types, align='center'),
    pn.Row(magnitude_slider, align='center'),
    pn.Row(depth_slider, align='center'),
    pn.Row(date_slider, align='center'), 
    pn.Row(plot_earthquake_panel, align='center')
 )


In [ ]:
# change value of 'plot_where' to:
# 'inline' if you would like the plot to show in the notebook
# 'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)

plot_where = 'pop-out'

**If the visualization is too slow, please follow the instructions in loading the data for taking a sample.**

After running the cell below you will have a panel with several widgets to index the eartquake data; by magnitude, depth and time, while the colors on the map show either the magintude or the depth of the earthquakes. 

In [ ]:
def show_earthquake(plot_where):
    if plot_where=='inline':
        return earthquake_panel
    elif plot_where=='pop-out':
        earthquake_panel.show()
    else:
        print('please use either inline or pop-out for the plot_where variable')

show_earthquake(plot_where);

### Explore the earthquake data & questions
Using the earthquake plot, try to answer the questions below. In order to view the questions, run the code cell.

In [ ]:
question_data11 = {
        "name": "Q1-1",
        "question": "At what depth do earthquakes along divergent boundaries generally occur?",
        "options": {
            "a": "less than 33 km",
            "b": "between 33 km and 74 km",
            "c": "between 74 km and 140 km",
            "d": "more than 140 km "
        },
        "answer": "a",
        "feedback": {"correct": "Correct, earthquakes at diverging boundaries typically occur at shallower depths",
                     "incorrect": "Incorrect. Note that at diverging boundaries the tectonic plates move away from each other. Magma wells up to (relatively) shallow depths in order to fill the gap. This may cause earthquakes. Where are these earthquakes likeliest to occur?"}
    }


question_data12 = {
        "name": "Q1-2",
        "question": "Where do earthquakes with a depth > 250 km typically occur? Multiple answers may be correct.",
        "options": {
            "a": "wherever two oceanic plates converge",
            "b": "at a boundary between an oceanic and continental plate",
            "c": "at island arcs",
            "d": "wherever two oceanic plates diverge"
        },
        "answers": ["a", "c"],
        "feedback": {"correct": "Correct, indeed very deep earthquakes occur  mostly when two ocaenic plates converge, or at an island arc.",
                     "incorrect": "Incorrect. Deep earthquakes occur due to deep processes such as subduction. Due to subduction, island arcs may form (see Figure 2.9 in the book). Where does subduction typically occur?"}
    }


question_data13 = {
        "name": "Q1-3",
        "question": "How should one set the depth slider (approximately) to capture the majority of the earthquakes at convergent boundaries?",
        "options": {
            "a": "0 - 33 km",
            "b": "33 - 250 km",
            "c": "250 - 350 km",
            "d": "350 - 650 km"
        },
        "answer": "b",
        "feedback": {"correct": "Correct, indeed most earthquakes at convergent boundaries occur between 33 km and 250 km depth.",
                     "incorrect": "Incorrect. Shallow earthquakes occur mostly at divergent boundaries (see also question 1). Earthquakes at a depth greater than 250 km do indeed occur at converging boundaries, but this range excludes a lot of earthquakes which do in fact occur at converging boundaries."}
    }


question_data14 = {
        "name": "Q1-4",
        "question": "How can one distinguish convergent boundaries from divergent boundaries using the earthquake information? Multiple answers may be correct.",
        "options": {
            "a": "width of earthquake zone",
            "b": "magnitude of earthquakes",
            "c": "earthquake depth",
            "d": "frequency of earthquakes"
        },
        "answers": ["a", "c", "d"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Try to use the sliders in the figure to find out at what depth earthquakes typically occur for different boundary types. By adjusting the date range, you can also check the frequency of earthquakes for different boundaries. Finally, look at the width of the earthquake zone and the magnitude of earthquakes for different types of boundaries. What differences do you observe?"}
    }


pn.Column(MC(question_data11), 
          MS(question_data12), 
          MC(question_data13), 
          MS(question_data14))

Earthquake data support one of the most fundamental processes in the geology: plate tectonics. Although plate tectonics is a relatively slow process that acts on the geological time scale, it has had an enormous impact on the formation of coastlines and determines the broadest features of the coast.

In [ ]:
question_data15 = {
        "name": "Q1-5",
        "question": "What are the two most important inherited aspects of the process of plate tectonics with respect to coastal features?",
        "options": {
            "a": "climate of the main land",
            "b": "latitude",
            "c": "continental shelf width",
            "d": "sediment supply"
        },
        "answers": ["c","d"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Read section 2.3.3. from the book, and try the question again."}
    }

pn.Column(MS(question_data15))

For the upcoming questions, consider the east Australian "Gold Coast".

In [ ]:
question_data16 = {
        "name": "Q1-6",
        "question": "Compare the shelf width of the Gold Coast with other coasts around Australia. Would you consider the continental shelf width to be relatively large or small for the Gold Coast?",
        "options": {
            "a": "large",
            "b": "small",
        },
        "answer": "b",
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Compare the Gold Coast with the northern, southern, and western Australian coast. The lighter blue color in the figure shows the shallower part of the ocean, i.e. the continental shelf."}
    }


question_data17 = {
        "name": "Q1-7",
        "question": "Is there a tectonic boundary close to the Gold Coast? If so, what is its type?",
        "options": {
            "a": "divergent",
            "b": "convergent",
            "c": "neither",
            "d": "no close boundary"
        },
        "answer": "b",
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. The closest boundary lies approximately beneath New Zealand. Use Figure 2.6 from the book to see the type boundary."}
    }


question_data18 = {
        "name": "Q1-8",
        "question": "Is there a tectonic boundary on the other side of the continent? If so, what is its type?",
        "options": {
            "a": "divergent",
            "b": "convergent",
            "c": "neither",
            "d": "no boundary"
        },
        "answer": "a",
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. The closest boundary lies in the southern Indian Ocean. Use Figure 2.6 from the book to see the type boundary."}
    }


question_data19 = {
        "name": "Q1-9",
        "question": "Is the Gold Coast protected by an island arc?",
        "options": {
            "a": "yes",
            "b": "no",
        },
        "answer": "a",
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Have a look at Figure 2.9 in the book, and the text below it. The Gold Coast is protected from the open ocean by New Zealand."}
    }


pn.Column(MC(question_data16), 
          MC(question_data17), 
          MC(question_data18), 
          MC(question_data19))

In 1971 Inman, D. L. & Nordstrom, C. E. used plate tectonics to classify the coast. They distinguish three main types of coasts: leading edge, trailing edge and marginal sea. In the questions below, match the correct characteristics to each class.

In [ ]:
question_data110 = {
        "name": "Q1-10",
        "question": "What are the typical characteristics of a leading edge coast? You may select multiple answers.",
        "options": {
            "a": "found at convergent tectonic boundary",
            "b": "found bordering ocean with diverging tectonic boundary",
            "c": "has three sub-types: neo, afro, and amero",
            "d": "coarse sediment",
            "e": "relatively deep earthquakes",
            "f": "frequent earthquakes",
            "g": "protected by landmass such as island arc",
            "h": "may contain large river deltas",
            "i": "found far away from tectonic boundaries",
            "j": "found close to tectonic boundaries",
            "k": "more volcanic activity",
            "l": "wide continental shelf",
        },
        "answers": ["a", "d", "e", "f", "j", "k"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Read section 2.3.2. and try again."}
    }


question_data111 = {
        "name": "Q1-11",
        "question": "What are the typical characteristics of a trailing edge coast? You may select multiple answers.",
        "options": {
            "a": "found at convergent tectonic boundary",
            "b": "found bordering ocean with diverging tectonic boundary",
            "c": "has three sub-types: neo, afro, and amero",
            "d": "coarse sediment",
            "e": "relatively deep earthquakes",
            "f": "frequent earthquakes",
            "g": "protected by landmass such as island arc",
            "h": "may contain large river deltas",
            "i": "found far away from tectonic boundaries",
            "j": "found close to tectonic boundaries",
            "k": "more volcanic activity",
            "l": "wide continental shelf",
        },
        "answers": ["b", "c", "h", "i", "l"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Read section 2.3.2. and try again."}
    }



question_data112 = {
        "name": "Q1-12",
        "question": "What are the typical characteristics of a marginal sea coast? You may select multiple answers.",
        "options": {
            "a": "found at convergent tectonic boundary",
            "b": "found bordering ocean with diverging tectonic boundary",
            "c": "has three sub-types: neo, afro, and amero",
            "d": "coarse sediment",
            "e": "relatively deep earthquakes",
            "f": "frequent earthquakes",
            "g": "protected by landmass such as island arc",
            "h": "may contain large river deltas",
            "i": "found far away from tectonic boundaries",
            "j": "found close to tectonic boundaries",
            "k": "more volcanic activity",
            "l": "wide continental shelf",
        },
        "answers": ["a", "g", "h", "j", "l"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Read section 2.3.2. and try again."}
    }


pn.Column(MS(question_data110), 
          MS(question_data111), 
          MS(question_data112))

Now that we have looked at the typical characteristics of each coastal type, we will look at some examples. Each question below specifies a different location. Use the earthquake panel to zoom to the locations specified, and classify the coasts to the correct classes. 

In [ ]:
question_data113 = {
        "name": "Q1-13",
        "question": "To what class of coasts does the coast at Wilmington (North Carolina, USA) belong?",
        "options": {
            "a": "leading edge",
            "b": "trailing edge",
            "c": "marginal sea",
        },
        "answer": "b",
        "feedback": {"correct": "Correct, this is indeed a trailing edge coast. Can you tell which sub-type of trailing edge coast it is?",
                     "incorrect": "Incorrect. The coast is not close to a converging boundary, nor is it protected by an island arc. The East American coast is relatively flat, and is home to large river deltas. What type of coast do these characteristics all suggest?"}
    }


question_data114 = {
        "name": "Q1-14",
        "question": "To what class of coasts does the coast at Shanghai (China) belong?",
        "options": {
            "a": "leading edge",
            "b": "trailing edge",
            "c": "marginal sea",
        },
        "answer": "c",
        "feedback": {"correct": "Correct, this is indeed a marginal sea coast, because though close to a converging boundary it is protected (by islands)?",
                     "incorrect": "Incorrect. The coast is close to a converging boundary, and it is protected by an island arc. Additionally, large river deltas are present (the Yangtze River delta for Shanghai specifically). What type of coast do these characteristics suggest?"}
    }


question_data115 = {
        "name": "Q1-15",
        "question": "To what class of coasts does the coast at Maputo (Mozambique) belong?",
        "options": {
            "a": "leading edge",
            "b": "trailing edge",
            "c": "marginal sea",
        },
        "answer": "b",
        "feedback": {"correct": "Correct, this is indeed a trailing edge coast. It is not close to a converging boundary so even though it is protected by an island (Madagascar), it is still a trailing edge coast.",
                     "incorrect": "Incorrect. The coast is not close to a converging boundary, which suggests what type of coast?"}
    }


question_data116 = {
        "name": "Q1-16",
        "question": "To what class of coasts does the coast at Lima (Peru) belong?",
        "options": {
            "a": "leading edge",
            "b": "trailing edge",
            "c": "marginal sea",
        },
        "answer": "a",
        "feedback": {"correct": "Correct, this is indeed a leading edge coast. It is close to a converging boundary and not sheltered by a landmass.",
                     "incorrect": "Incorrect. The coast is close to a converging boundary and not sheltered by a landmass, which suggests what type of coast?"}
    }


pn.Column(MC(question_data113), 
          MC(question_data114), 
          MC(question_data115), 
          MC(question_data116))

Inman, D. L. & Nordstrom (1971) further distinguish Afro-trailing-edge coasts and Amero-trailing-edge coasts based on differences in sediment supplies.

In [ ]:
question_data117 = {
        "name": "Q1-17",
        "question": "What are the main causes for these differences in sediment supply? You may select multiple answers.",
        "options": {
            "a": "topography",
            "b": "soil type",
            "c": "climate",
            "d": "vegetation type"
        },
        "answers": ["a", "c"],
        "feedback": {"correct": "Correct, topography and climate are indeed the main factors determining the sediment supply.",
                     "incorrect": "Incorrect. Read the section on 'sediment availability' from section 2.3.3. from the book and try again."}
    }


question_data118 = {
        "name": "Q1-18",
        "question": "Amero-trailing edge coasts receive less/more (choose) sediment from rivers compared to Afro-trailing edge coasts. The deltas formed at Amero-trailing edge coasts are therefore smaller/larger (choose) when compared to Afro-trailing edge coasts.",
        "options": {
            "a": "less",
            "b": "more",
            "c": "smaller",
            "d": "larger"
        },
        "answers": ["b", "d"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. Read the section on 'sediment availability' from section 2.3.3. from the book and try again."}
    }


question_data119 = {
        "name": "Q1-19",
        "question": "Where do we find the largest river deltas? You may select multiple answers.",
        "options": {
            "a": "eastern USA",
            "b": "western Europe",
            "c": "western Africa",
            "d": "south-eastern Asia",
            "e": "Brazil",
            "f": "southern Australia"
        },
        "answers": ["a", "d", "e"],
        "feedback": {"correct": "Correct.",
                     "incorrect": "Incorrect. What coastal type leads to the largest deltas forming?"}
    }


pn.Column(MS(question_data117), 
          MS(question_data118), 
          MS(question_data119))

####################################################

TO BE REMOVED

1) How do the earthquake magnitude and earthquake depth relate to the coasts that we see? (Hint: See Figure 2.3 in the textbook and consider how deep under the ground the plates are moving. Extra hint: How do earthquake magnitude and depth differ for convergent and divergent plate boundaries?)

2) Earthquake data support one of the most fundamental processes in the geology: plate tectonics. Although plate tectonics is a relatively slow process that acts on the [geological time scale](https://cdn.britannica.com/67/73167-050-B9A74092/chart.jpg), it has had an enormous impact on the formation of coastlines and determines the broadest features of the coast. What are two important inherited aspects of this process? (Hint: see Figure 2.10 and Sec. 2.3.3 in the textbook.) 

3) In 1971 Inman, D. L. & Nordstrom, C. E. used plate tectonics to classify the coast. Explain the classification that they introduced. What are the three different classes that they distinguish? How do they match with the earthquake data as you can explore in the panel? 

4) Can you identify or predict areas around the world where you will find the coasts that are distinguished by Inman, D. L. & Nordstrom, C. E.? For instance, what kind of coasts do you have in Chili? And how are they different to the east coast of the USA? And what is characteristic about the East China sea? 

5) Inman, D. L. & Nordstrom (1971) further distinguish Afro-trailing-edge coasts and Amero-trailing-edge coasts based on differences in sediment supplies. What is the main cause of these differences in sediment supply? And how do you expect the differences in sediment input to show in the coastal geomorphology?
####################################################

## Part 2: Process-based coastal classification

In the section part of this notebook we will explore several coastal systems around the world considering the second and third order scale of features. In chapter two of the Coastal Dynamics open textbook it is explained how coastal systems can be classified according to the processes that characterize these systems. For example, one of the figures (below) shows how the relative influence of fluvial, wave and tidal processes influences the shape of coastal features. The idea of this exercise is that you identify the signatures of the processes that are introduced in chapter 2 in several coastal systems around the world. 

![image](./figures/01_coastal_forcing.png)

### The coastal systems data

In the cell below we define a small plot function that generates a ESRI World Imagery basemap given a longitude, latitude, zoom level and name. Also, a small sheet of coastal systems around the world is loaded into `geopandas`, a Python library for geospatial tabular data. In the cells afterwards we sample this dataframe and show the coastal system on a map. Since the sample is random you might encounter the same coastal system multiple times; then you can just run the cell again to get another 'coastal draw'.  


In [ ]:
coastal_systems = gpd.read_file(coastal_systems_fp)

In [ ]:
pn.extension()
title_bar = pn.Row(
    pn.pane.Markdown(
        "## Exercise 2: Coastal system characterization",
        styles={"color": "black"},
        width=800,
        sizing_mode="fixed",
        margin=(10, 5, 10, 15),
    ),
    pn.Spacer(),
)

options = coastal_systems.name.to_list()
coastal_systems_slider = pn.widgets.Select(
    name="Coastal system", options=options, value=np.random.choice(options)
)


@pn.depends(coastal_systems_slider.param.value)
def plot_coastal_system(name):
    system = coastal_systems.loc[coastal_systems["name"] == name].copy()
    west, south, east, north = system[
        ["west", "south", "east", "north"]
    ].values.flatten()

    p = system.hvplot.points(
        x="lon",
        y="lat",
        geo=True,
        color="red",
        alpha=0,
        xlim=(west, east),
        ylim=(south, north),
        tiles="ESRI",
        frame_width=1100,
        ylabel="Latitude [deg]",
        xlabel="Longitude [deg]",
    )

    p.opts(frame_width=1000, frame_height=500, tools=['pan', 'wheel_zoom'])

    return p


app = pn.Column(
    # title_bar,
    pn.Row(coastal_systems_slider, align='center'),
    pn.Row(plot_coastal_system, align='center'),
)

After running the cell below you will have a panel showing sattelite images of different coastal systems.

In [ ]:
# change value of 'plot_where' to:
# 'inline' if you would like the plot to show in the notebook
# 'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)

plot_where = 'pop-out'

In [ ]:
def show_coastal_system(plot_where):
    if plot_where=='inline':
        return app
    elif plot_where=='pop-out':
        app.show()
    else:
        print("please use either 'inline' or 'pop-out' for the 'plot_where' variable")

show_coastal_system(plot_where);

Using this panel, try to answer the questions below. The questions are seperated by 'theory' and 'application'.

### Questions 2a: theory


In [ ]:
# Questions 2a

#### Write draft for questions 2a here.
What is the tectonic plate setting for system ... and ... respectively?

Estimate the order of magnitude of the width of the continental shelf for system ....

Which is the nearest plate boundary for system ...?

What is the coastal type for (some system)? And for (some system)?

(some system) is a trailing edge coast. Is it an Amero-, Afro-, or neo-trailing edge coast?

Where does system ... get its sediment supply from?

What is the dominant process for system? Choose the best-fitting option.

Look at system .... Does feature X demonstrate wave or tidal influence?

Look at system .... How can we see that the system is dominated by waves/river/tides?

For system ..., we observe the presence of vegetation. Where is this vegetation mostly located?
- tidal flats
- river bed
- seashore
- ...

Where do you estimate ... to be positioned in figure 2.44?

Where do you estimate ... to be positioned in fifgure 2.44?

Look at the examples of different coastal systems in figure 2.42 in the book. Then look at system .... Which example number shows the most resemblence? And for system ...?

What is the name of (some feature) for system ...?

### Questions 2b: application

In [ ]:
# Questions 2b

#### Write draft for questions 2b here.
Which is an example of a highly engineered river-dominated delta? (different options)

Which is an example of a more natural river-dominated delta? (different options)

What is the difference in scale for ... and ...?

The ... and ... are wave dominated. Which has a higher sediment supply? This is because of ...

Difference between large sandy sediment supply and large mud supply can be seen from ...

The eastern and western tips of the Dutch and German Wadden Islands are very different beach ridge environments. This can be explained by differences in sediment supply. From which direction is the sediment coming in?

The Dune du Pilat in France is one of the world's largest coastal sand dunes (it is also one of the coolest and you should definitely visit if you get the chance!). It is located on the east side of Arcachon Inlet. Select the reason(s) that it is located on the east side (and not the west).

The northern Jiangsu coast in China has a limited sediment supply. Select the statement which best exlpains this.

Both ... and ... have extensive tidal flats. However, only ... has mangroves and/or salt marshes. Select the best fitting reason for this.

... has jetties. How do you expect the inlet to evolve in the future? Select the best fitting answer.

Rocky coastal features might constrain a coastal system from evolving a certain way. For which of the following coastal systems is this relevant?

The Albufeira Lagoon in Portugal opens and closes seasonally. In the image shown, is it open or closed? When do you expect the lagoon to be open?

Look at .... In the past, this was a natural system with multiple channels forming a delta. Nowadays, the area is heavily urbanized and the system has been linearized. Has the tidal influence increased or decreased due to these human interventions?

Based on these satellite images, which is the most beautiful site? 
Good choice! Taking a moment to appreciate the beauty of these natural systems is an important part of your job as coastal engineers.

This is the end of the notebook for week 1. We discussed tectonic and process-based classification of coastal systems. For further reading, see chapter 2 of the book. We hope you had fun!

############################################################################
TO BE REMOVED

### Explore the coastal systems

While sampling over a range of coastal systems, try to answer the following questions. 

1. Find and compare a heavily engineered river-dominated delta and a more natural river-dominated delta
2. Compare the scale of the biggest and smallest tidal basin in the dataset
3. Find the estuarine and deltaic systems with a spit
4. Compare and contrast wave-dominated deltas with high and low sediment supply. How can you tell?
5. Find a tidal estuary with large fine (muddy) sediment supply, then find one with a large coarse (sandy) sediment supply. How can you tell the difference?
6. The eastern and western tips of the Dutch and German Wadden Islands are very different beach ridge environments. How might differences in sediment supply explain this? Where is the sediment coming from?
7. The Dune du Pilat in France is one of the world's largest coastal sand dunes (it is also one of the coolest and you should definitely visit if you get the chance!). Why is it located on the east side of Arcachon Inlet and not the west?
8. Look at the northern Jiangsu coast in China. What might explain the limited sediment supply in this location?
9. Find an estuary or tidal bay with extensive intertidal flats. Do you see salt marshes or mangrove forests nearby?  Why or why not?
10. Find an inlet with jetties. How might this affect the way it evolves?
11. Find a delta/estuary/inlet whose shape is constrained by the presence of rocky coastal features.  
12. The Albufeira Lagoon in Portugal opens and closes seasonally. In the image shown, is it open or closed? When and how might it open or close? 
13. Find examples of heavily urbanized estuaries. How might these human interventions influence the natural processes there?
14. Based on these satellite images, which is the most beautiful site? Taking a moment to appreciate the beauty of these natural systems is an important part of your job as coastal engineers.

############################################################################